In [1]:
import os
import itertools
import functools
import tomli
import pathlib
import glob
import re
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy as sc
import mne
import mne_nirs

In [2]:
import utils
import constants
from nirs import NIRS

In [3]:
NIRS().read(1, 1, 3)

Creating RawArray with float64 data, n_channels=36, n_times=12333
    Range : 0 ... 12332 =      0.000 ...   891.896 secs
Ready.
